# Differential Expression Analysis

This notebook is a first attempt to automate differential gene expression analysis with Python. 

Gene expression data are read from the .tsv file and imported into Python data structures so that we can select "interesting" genes to submit to Enrichr for enrichment analysis.

We'll follow these steps:
1. Filter the most significant genes (p<0.01)
2. Filter Log2FC (Less and equal to -1 or Great and equal to 1P)
3. Clean up data and maintain only Entrez and Ensemble ID, Log2 FC, p.value cols
4. Rank the genes (largest to  smallest using Log2FC)
5. Past results on Enrichr

In [1]:
# Import necessary Python libraries

# Data analysis
import numpy as np
import pandas as pd

# Bioinformatics
import gseapy as gp

# Graphics and plotting
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2

#import seaborn as sns

## Data preparation

In [2]:
# Reading in data from tsv file (the "Feature ID" column will be used as id)

filename = '../data/Merged_differential_expression_results.tsv'

dex = pd.read_csv(filename, index_col=0, sep='\t')

In [3]:
# Separate the columns into strings (all gene names, descriptions, ...) and numeric (gene expressions, fold changes, and quality metrics)
all_cols = list(dex.columns)
descr_cols = ['Feature_ID', 'entrezgene', 'external_gene_name', 'gene_biotype', 'external_gene_source', 'description']
numeric_cols = [x for x in all_cols if x not in set(descr_cols)]

# The select samples, fold metrics
sample_cols = [x for x in numeric_cols if 'sample' in x]
linfold_cols = [x for x in numeric_cols if 'linearFC' in x]
logfold_cols = [x for x in numeric_cols if 'logFC' in x]

control_3h_cols = ['sample.'+str(x) for x in range(1,5)]
control_12h_cols = ['sample.'+str(x) for x in range(5,9)]
CC_3h_cols = ['sample.'+str(x) for x in range(9,13)]
CC_12h_cols = ['sample.'+str(x) for x in range(13,17)]
BL21_3h_cols = ['sample.'+str(x) for x in range(17,21)]
BL21_3h_bcols = ['sample.'+str(x) for x in range(21,25)]
LPS_3h_cols = ['sample.'+str(x) for x in range(25,29)]
LPS_12h_cols = ['sample.'+str(x) for x in range(29,33)]

# Associate fold column name with p-value column name
adj_p_value_columns = {
    'BL21_12hr-BL21_3hr_logFC': 'BL21_12hr-BL21_3hr_adj.P.Val',
    'BL21_12hr-control_12hr_logFC': 'BL21_12hr-BL21_3hr_adj.P.Val',
    'BL21_3hr-control_3hr_logFC': 'BL21_3hr-control_3hr_adj.P.Val',
    'CC_12hr-CC_3hr_logFC': 'CC_12hr-CC_3hr_adj.P.Val',
    'CC_12hr-control_12hr_logFC':'CC_12hr-control_12hr_adj.P.Val',
    'CC_3hr-control_3hr_logFC':'CC_3hr-control_3hr_adj.P.Val',
    'control_12hr-control_3hr_logFC':'control_12hr-control_3hr_adj.P.Val',
    'LPS_12hr-control_12hr_logFC':'LPS_12hr-control_12hr_adj.P.Val',
    'LPS_12hr-LPS_3hr_logFC':'LPS_12hr-LPS_3hr_adj.P.Val',
    'LPS_3hr-control_3hr_logFC':'LPS_3hr-control_3hr_adj.P.Val'
}

### Selecting data

In [4]:
# Change this value to the column of interest; at the moment, it must be one of the columns storing the fold changes,
# e.g.: control_12hr-control_3hr_logFC
data_column_name = 'CC_3hr-control_3hr_logFC'
pval_column_name = adj_p_value_columns[data_column_name]

# Select only significant p-values (rows for which p-value is less than a threshold, e.g.: p<0.01)
data = dex[dex[pval_column_name]<0.05]

# Select only LogFC values which are <=-1 or >=1
# data = dex[(dex[data_column_name]<=-1) | (dex[data_column_name]>=1)]

# get rid of all columns we are not interested in right now
#data = data[['external_gene_name', data_column_name]]
data = data[['Feature_ID', data_column_name]]

In [5]:
# Now sort rows by ascending expression value
data = data.sort_values(
    by=data_column_name,
    ascending=False
    )

In [ ]:
#data.to_csv('../data/'+data_column_name+'.rnk', index=False, sep='\t', header=['geneID', 'logFC'])
data.to_csv('../data/'+data_column_name+'.rnk', index=False, sep='\t', header=False)

In [6]:
data.describe()

,CC_3hr-control_3hr_logFC
count,121.000000
mean,0.001413
std,0.394483
min,-0.860053
25%,-0.284765
50%,-0.182841
75%,0.251556
max,1.571771
